Project overview:
The target is to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data are transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information  available in as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

I will create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

Parameter	Value
Data URL	https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks
Exchange rate CSV path	https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Table Attributes (upon Extraction only)	Name, MC_USD_Billion
Table Attributes (final)	Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion
Output CSV Path	./Largest_banks_data.csv
Database name	Banks.db
Table name	Largest_banks
Log file	code_log.txt

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

# Set the search parameters
filename = 'exchange_rate.csv'
# take only inputs from above excel , all computations should be in py only
#equivalent file excel weights is (Final) Updated Weights_Dometic 2022_1 september-added-corrected
search_path = '/content/drive'

# Search for the file
for root, dirs, files in os.walk(search_path):
    if filename in files:
        file_path = os.path.join(root, filename)
        break
file_path
dexchange_rate = pd.read_csv(file_path)
dexchange_rate

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [6]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime

Project tasks
Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

In [7]:
def log_progress(message):
  timevar='%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
  now=datetime.now()
  timestamp=now.strftime(timevar)
  with open('./code_log.txt','a') as f:
    f.write(timestamp+' : ' + message + '\n')

In [9]:
url='https://en.wikipedia.org/wiki/List_of_largest_banks'

Task 2:
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

In [10]:
def extract(url):
  page=requests.get(url).text
  tabel=BeautifulSoup(page,'html.parser')
  df=pd.DataFrame(columns=['Name', 'MC_USD_Billion'])
  table_row=tabel.find_all(name='tbody')
  rows=table_row[0].find_all(name='tr')# 0 as first table
  for i,row in enumerate(rows):
    col=row.find_all(href=True)
    if len(col)>1:
      if 1==1:
        dict={'Name':col[1].contents[0],'MC_USD_Billion':row.find_all(name='td')[2].contents[0]}
        print(col[1].contents[0])
        df1=pd.DataFrame(dict,index=[0])
        df=pd.concat([df,df1], ignore_index=True,axis=0)
  return df
df=extract(url)

JPMorgan Chase
Bank of America
Industrial and Commercial Bank of China
Agricultural Bank of China
HDFC Bank
Wells Fargo
HSBC Holdings PLC
Morgan Stanley
China Construction Bank
Bank of China


Task 3:
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

In [11]:
def transform(df):
  tst=df['MC_USD_Billion'].to_list()
  tst=[w.replace('\n','') for w in tst]
  tst=[float(w) for w in tst]
  df['MC_USD_Billion']=pd.DataFrame(tst)
  df['MC_GBP_Billion']=df['MC_USD_Billion']*.8
  df['MC_EUR_Billion']=df['MC_USD_Billion']*.93
  df['MC_INR_Billion']=df['MC_USD_Billion']*82.95
  return df
transform(df)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,419.25,335.400,389.9025,34776.7875
1,Bank of America,231.52,185.216,215.3136,19204.5840
2,Industrial and Commercial Bank of China,194.56,155.648,180.9408,16138.7520
3,Agricultural Bank of China,160.68,128.544,149.4324,13328.4060
4,HDFC Bank,157.91,126.328,146.8563,13098.6345
5,Wells Fargo,155.87,124.696,144.9591,12929.4165
6,HSBC Holdings PLC,148.90,119.120,138.4770,12351.2550
7,Morgan Stanley,140.83,112.664,130.9719,11681.8485
8,China Construction Bank,139.82,111.856,130.0326,11598.0690
9,Bank of China,136.81,109.448,127.2333,11348.3895


Task 4:
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

In [12]:
def load_to_csv(df,path):
  df.to_csv(path)


In [13]:
conn = sqlite3.connect('STAFF.db')
table_name='Countries_by_GDP'
def load_to_db(df,conn,table_name):
  df.to_sql(table_name,conn,if_exists='replace',index=True)
  return table_name

load_to_db(df,conn,table_name)

'Countries_by_GDP'

Task 5:
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

In [15]:
def run_query(query,conn,table_name):
  query_output=pd.read_sql(query,conn)
  print(query)
  print(query_output)
#run_query(query,conn,table_name)

Task 6:
Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.




In [16]:
url='https://en.wikipedia.org/wiki/List_of_largest_banks'
path='./Largest_banks_data.csv'
log_progress('Preliminaries complete. Initiating ETL process')
df=extract(url)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df,path)
log_progress('Data saved to CSV file')
load_to_db(df, conn, table_name)
log_progress('Data loaded to Database as table. Running the query')
query=f'select * from {table_name}'
run_query(query,conn,table_name)
log_progress('Process Complete.')

JPMorgan Chase
Bank of America
Industrial and Commercial Bank of China
Agricultural Bank of China
HDFC Bank
Wells Fargo
HSBC Holdings PLC
Morgan Stanley
China Construction Bank
Bank of China
select * from Countries_by_GDP
   index                                     Name  MC_USD_Billion  \
0      0                           JPMorgan Chase          419.25   
1      1                          Bank of America          231.52   
2      2  Industrial and Commercial Bank of China          194.56   
3      3               Agricultural Bank of China          160.68   
4      4                                HDFC Bank          157.91   
5      5                              Wells Fargo          155.87   
6      6                        HSBC Holdings PLC          148.90   
7      7                           Morgan Stanley          140.83   
8      8                  China Construction Bank          139.82   
9      9                            Bank of China          136.81   

   MC_GBP_Billion 

In [19]:
print('./code_log.txt')
with open('./code_log.txt') as logfile:
  for row in logfile:
    print(row)

./code_log.txt
2023-Nov-11-13:46:33 : Preliminaries complete. Initiating ETL process

2023-Nov-11-13:46:33 : Data extraction complete. Initiating Transformation process

2023-Nov-11-13:46:33 : Data transformation complete. Initiating loading process

2023-Nov-11-13:46:33 : Data saved to CSV file

2023-Nov-11-13:46:33 : Data loaded to Database as table. Running the query

2023-Nov-11-13:46:33 : Process Complete.

